# __Predicting Outcomes of Call Option Contracts__

# SECTION 1: Preparation

##  1.1.) Loading the python packages

In [1]:

from sklearn import datasets

# Import the modules
import numpy as np
import pandas as pd
from pathlib import Path
import random

# Load libraries
import numpy as np
import pandas as pd
from matplotlib import pyplot
from pandas import read_csv, set_option
from pandas.plotting import scatter_matrix
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold, cross_val_score, GridSearchCV
from evolutionary_search import EvolutionaryAlgorithmSearchCV

import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.tree import DecisionTreeClassifier
#from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
#from sklearn.neural_network import MLPClassifier
from sklearn.pipeline import Pipeline
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier, RandomForestClassifier, ExtraTreesClassifier

#importing classification_report, confusion_matrix
from sklearn.metrics import classification_report, confusion_matrix

#importing accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

# import packages for hyperparameters tuning
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe

#from sklearn.metrics import balanced_accuracy_score
#from sklearn.metrics import confusion_matrix
#from imblearn.metrics import classification_report_imbalanced

#Libraries for Saving the Model
from pickle import dump
from pickle import load

import warnings
warnings.filterwarnings('ignore')



## 1.2.) Loading the Data

In [2]:
#NOTE: This iris dataset is for debugging purposes only

#iris = datasets.load_iris()

#X = iris.data

#y = iris.target

#validation_size = 0.2

#seed = 7

In [3]:
#Import call options info

df = pd.read_csv(
    Path("../Resources/test.csv")
)


In [4]:
df.head()

,QUOTE_READTIME,SPY PRICE,EXPIRE_DATE,DTE,C_DELTA,C_GAMMA,C_VEGA,C_THETA,C_RHO,C_IV,...,STRIKE_DISTANCE_PCT,STRIKE DISTANCE,PRICECLOSE,ROI %,y,VIX PRICE,AVG SCORE,MED SCORE,SEN IND AVG NUM,SEN IND MED NUM
0,2022-02-24 14:30:00,420.68,2022-02-28,4.06,0.67743,0.02409,0.16594,-0.45141,0.02360,0.37685,...,0.021,-8.68,NaN,NaN,NaN,32.01,-0.044775,0.0,0,0
1,2022-02-24 14:30:00,420.68,2022-02-28,4.06,0.65631,0.02476,0.17042,-0.46023,0.02290,0.36946,...,0.018,-7.68,NaN,NaN,NaN,32.01,-0.044775,0.0,0,0
2,2022-02-24 14:30:00,420.68,2022-02-28,4.06,0.63414,0.02544,0.17446,-0.46849,0.02288,0.36332,...,0.016,-6.68,NaN,NaN,NaN,32.01,-0.044775,0.0,0,0
3,2022-02-24 14:30:00,420.68,2022-02-28,4.06,0.69735,0.02335,0.16063,-0.44086,0.02317,0.38448,...,0.023,-9.68,NaN,NaN,NaN,32.01,-0.044775,0.0,0,0
4,2022-02-24 15:00:00,423.38,2022-02-28,4.04,0.63464,0.02711,0.17433,-0.43781,0.02282,0.34336,...,0.015,-6.38,NaN,NaN,NaN,30.86,-0.044775,0.0,0,0


# SECTION 2: Exploratory Data Analysis

## 2.1.) Descriptive Statistics

In [5]:
df.shape

(2900, 23)

In [6]:
#Check for any null values and remove the null values
print('Null Values =', df.isnull().values.any())

#Drop NaNs
df.dropna()

Null Values = True


,QUOTE_READTIME,SPY PRICE,EXPIRE_DATE,DTE,C_DELTA,C_GAMMA,C_VEGA,C_THETA,C_RHO,C_IV,...,STRIKE_DISTANCE_PCT,STRIKE DISTANCE,PRICECLOSE,ROI %,y,VIX PRICE,AVG SCORE,MED SCORE,SEN IND AVG NUM,SEN IND MED NUM
310,2022-02-28 09:30:00,432.00,2022-03-04,4.27,0.28399,0.02206,0.17083,-0.52948,0.01656,0.30466,...,0.021,9.00,2.39,-11.152416,1.0,31.91,-0.042974,0.0,0,0
311,2022-02-28 09:30:00,432.00,2022-03-04,4.27,0.25895,0.02137,0.16419,-0.49771,0.01529,0.30035,...,0.023,10.00,1.90,-43.952802,1.0,31.91,-0.042974,0.0,0,0
312,2022-02-28 09:30:00,432.00,2022-03-04,4.27,0.23477,0.02056,0.15547,-0.46198,0.01378,0.29565,...,0.025,11.00,1.51,-48.287671,1.0,31.91,-0.042974,0.0,0,0
313,2022-02-28 09:30:00,432.00,2022-03-04,4.27,0.14922,0.01642,0.11821,-0.32177,0.00827,0.28013,...,0.035,15.00,0.56,-47.169811,1.0,31.91,-0.042974,0.0,0,0
314,2022-02-28 09:30:00,432.00,2022-03-04,4.27,0.18994,0.01865,0.13795,-0.39168,0.01146,0.28738,...,0.030,13.00,0.95,-37.500000,1.0,31.91,-0.042974,0.0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2895,2022-03-16 11:00:00,433.31,2022-03-18,2.21,0.84058,0.02339,0.08571,-0.34777,0.01188,0.42580,...,0.033,-14.31,15.33,6.532314,2.0,27.58,0.089130,0.0,0,0
2896,2022-03-16 11:00:00,433.31,2022-03-18,2.21,0.83081,0.02509,0.08944,-0.35051,0.01248,0.40984,...,0.031,-13.31,15.00,11.275964,3.0,27.58,0.089130,0.0,0,0
2897,2022-03-16 11:00:00,433.31,2022-03-18,2.21,0.69645,0.03227,0.12836,-0.52916,0.01394,0.35986,...,0.017,-7.31,9.67,11.021814,3.0,27.58,0.089130,0.0,0,0
2898,2022-03-16 11:00:00,433.31,2022-03-18,2.21,0.78689,0.02699,0.10522,-0.43651,0.01315,0.39882,...,0.026,-11.31,13.11,10.913706,3.0,27.58,0.089130,0.0,0,0


In [7]:
display(df.head())
df.tail()

,QUOTE_READTIME,SPY PRICE,EXPIRE_DATE,DTE,C_DELTA,C_GAMMA,C_VEGA,C_THETA,C_RHO,C_IV,...,STRIKE_DISTANCE_PCT,STRIKE DISTANCE,PRICECLOSE,ROI %,y,VIX PRICE,AVG SCORE,MED SCORE,SEN IND AVG NUM,SEN IND MED NUM
0,2022-02-24 14:30:00,420.68,2022-02-28,4.06,0.67743,0.02409,0.16594,-0.45141,0.02360,0.37685,...,0.021,-8.68,NaN,NaN,NaN,32.01,-0.044775,0.0,0,0
1,2022-02-24 14:30:00,420.68,2022-02-28,4.06,0.65631,0.02476,0.17042,-0.46023,0.02290,0.36946,...,0.018,-7.68,NaN,NaN,NaN,32.01,-0.044775,0.0,0,0
2,2022-02-24 14:30:00,420.68,2022-02-28,4.06,0.63414,0.02544,0.17446,-0.46849,0.02288,0.36332,...,0.016,-6.68,NaN,NaN,NaN,32.01,-0.044775,0.0,0,0
3,2022-02-24 14:30:00,420.68,2022-02-28,4.06,0.69735,0.02335,0.16063,-0.44086,0.02317,0.38448,...,0.023,-9.68,NaN,NaN,NaN,32.01,-0.044775,0.0,0,0
4,2022-02-24 15:00:00,423.38,2022-02-28,4.04,0.63464,0.02711,0.17433,-0.43781,0.02282,0.34336,...,0.015,-6.38,NaN,NaN,NaN,30.86,-0.044775,0.0,0,0


,QUOTE_READTIME,SPY PRICE,EXPIRE_DATE,DTE,C_DELTA,C_GAMMA,C_VEGA,C_THETA,C_RHO,C_IV,...,STRIKE_DISTANCE_PCT,STRIKE DISTANCE,PRICECLOSE,ROI %,y,VIX PRICE,AVG SCORE,MED SCORE,SEN IND AVG NUM,SEN IND MED NUM
2895,2022-03-16 11:00:00,433.31,2022-03-18,2.21,0.84058,0.02339,0.08571,-0.34777,0.01188,0.42580,...,0.033,-14.31,15.33,6.532314,2.0,27.58,0.08913,0.0,0,0
2896,2022-03-16 11:00:00,433.31,2022-03-18,2.21,0.83081,0.02509,0.08944,-0.35051,0.01248,0.40984,...,0.031,-13.31,15.00,11.275964,3.0,27.58,0.08913,0.0,0,0
2897,2022-03-16 11:00:00,433.31,2022-03-18,2.21,0.69645,0.03227,0.12836,-0.52916,0.01394,0.35986,...,0.017,-7.31,9.67,11.021814,3.0,27.58,0.08913,0.0,0,0
2898,2022-03-16 11:00:00,433.31,2022-03-18,2.21,0.78689,0.02699,0.10522,-0.43651,0.01315,0.39882,...,0.026,-11.31,13.11,10.913706,3.0,27.58,0.08913,0.0,0,0
2899,2022-03-16 11:00:00,433.31,2022-03-18,2.21,0.76485,0.02798,0.11177,-0.47164,0.01364,0.39213,...,0.024,-10.31,11.94,8.348457,2.0,27.58,0.08913,0.0,0,0


In [8]:
df.describe()

,SPY PRICE,DTE,C_DELTA,C_GAMMA,C_VEGA,C_THETA,C_RHO,C_IV,C_VOLUME,C_LAST,...,STRIKE_DISTANCE_PCT,STRIKE DISTANCE,PRICECLOSE,ROI %,y,VIX PRICE,AVG SCORE,MED SCORE,SEN IND AVG NUM,SEN IND MED NUM
count,2900.000000,2900.000000,2900.000000,2900.000000,2900.000000,2900.000000,2900.000000,2900.000000,2900.000000,2900.000000,...,2900.000000,2900.000000,2086.000000,2086.000000,2086.000000,2900.000000,2900.000000,2900.000000,2900.000000,2900.000000
mean,428.951103,3.304966,0.328212,0.020871,0.117730,-0.403352,0.012343,0.285923,2340.001379,4.038745,...,0.024346,5.948897,4.470000,53.773392,2.710930,31.791097,0.004794,0.048436,0.086897,0.086897
std,6.599468,0.731205,0.258034,0.009059,0.054512,0.222303,0.008285,0.050773,3464.616078,5.310466,...,0.016121,10.982615,4.352018,109.686176,1.662492,2.159630,0.157967,0.157037,0.281732,0.281732
min,415.920000,2.000000,0.001940,0.000730,0.002830,-0.860310,-0.000260,0.158170,0.000000,0.010000,...,0.000000,-29.660000,0.060000,-93.258427,0.000000,27.120000,-0.167307,0.000000,0.000000,0.000000
25%,423.200000,3.020000,0.123460,0.014157,0.073510,-0.588215,0.005430,0.264770,325.750000,0.710000,...,0.011000,0.560000,0.950000,-19.507028,1.000000,30.540000,-0.123090,0.000000,0.000000,0.000000
50%,429.510000,3.190000,0.269585,0.022935,0.127440,-0.412835,0.011480,0.292975,1162.500000,2.070000,...,0.022500,7.430000,2.840000,10.089409,2.500000,31.710000,-0.042974,0.000000,0.000000,0.000000
75%,434.700000,4.060000,0.471875,0.027312,0.162665,-0.216868,0.017997,0.315408,3059.500000,5.000000,...,0.036000,13.330000,7.000000,101.607506,5.000000,33.310000,0.076774,0.000000,0.000000,0.000000
max,440.340000,4.270000,0.961580,0.039320,0.205640,-0.003750,0.035810,0.447280,48413.000000,29.660000,...,0.071000,29.620000,16.460000,487.719298,5.000000,37.030000,0.445772,0.557400,1.000000,1.000000


In [9]:
df.dtypes

QUOTE_READTIME          object
SPY PRICE              float64
EXPIRE_DATE             object
DTE                    float64
C_DELTA                float64
C_GAMMA                float64
C_VEGA                 float64
C_THETA                float64
C_RHO                  float64
C_IV                   float64
C_VOLUME               float64
C_LAST                 float64
STRIKE                 float64
STRIKE_DISTANCE_PCT    float64
STRIKE DISTANCE        float64
PRICECLOSE             float64
ROI %                  float64
y                      float64
VIX PRICE              float64
AVG SCORE              float64
MED SCORE              float64
SEN IND AVG NUM          int64
SEN IND MED NUM          int64
dtype: object

## 2.2.) Feature Analysis and Exploration

### Plotting features according to contract length

In [10]:
#
#contract_outcome = df.groupby('STRIKE_DISTANCE_PCT')['y'].value_counts(normalize=True).loc[:,1]
#sns.set(rc={'figure.figsize':(12,5)})
#sns.barplot(x=contract_outcome.index, y=contract_outcome.values, color='#5975A4', saturation=1)

### Eliminate Uncorrelated Features

In [11]:
#Calculate correlation of each feature with 'y'

correlation = df.corr()
correlation_df = abs(correlation['y'])

In [12]:
correlation_df.sort_values(ascending=False)

y                      1.000000
ROI %                  0.864399
STRIKE                 0.617536
PRICECLOSE             0.604456
SPY PRICE              0.501226
C_IV                   0.432319
AVG SCORE              0.281606
C_VEGA                 0.207979
STRIKE_DISTANCE_PCT    0.185971
STRIKE DISTANCE        0.159732
C_DELTA                0.141756
C_THETA                0.136926
C_RHO                  0.130084
C_LAST                 0.128625
C_GAMMA                0.097552
DTE                    0.097524
VIX PRICE              0.058291
C_VOLUME               0.027472
MED SCORE                   NaN
SEN IND AVG NUM             NaN
SEN IND MED NUM             NaN
Name: y, dtype: float64

In [13]:
#Drop variables with less than 3% correlation with contract outcome ('y')

#drop_list_corr = sorted(list(correlation_df[correlation_df < 0.03].index))
#print(drop_list_corr)

In [14]:
#Dropping because C_IV convolutes outcome for some buggish reason

df.drop(labels=["C_IV"], axis=1, inplace=True)

### Drop Columns Not Needed for Machine Learning

In [15]:
df.drop(columns=["QUOTE_READTIME"], inplace=True)

In [16]:
df.drop(columns=["ROI %"], inplace=True)

In [17]:
df.drop(columns=["PRICECLOSE"], inplace=True)

In [18]:
df.drop(columns=["EXPIRE_DATE"], inplace=True)

In [19]:
df

,SPY PRICE,DTE,C_DELTA,C_GAMMA,C_VEGA,C_THETA,C_RHO,C_VOLUME,C_LAST,STRIKE,STRIKE_DISTANCE_PCT,STRIKE DISTANCE,y,VIX PRICE,AVG SCORE,MED SCORE,SEN IND AVG NUM,SEN IND MED NUM
0,420.68,4.06,0.67743,0.02409,0.16594,-0.45141,0.02360,1744.0,10.72,412.0,0.021,-8.68,NaN,32.01,-0.044775,0.0,0,0
1,420.68,4.06,0.65631,0.02476,0.17042,-0.46023,0.02290,1855.0,8.76,413.0,0.018,-7.68,NaN,32.01,-0.044775,0.0,0,0
2,420.68,4.06,0.63414,0.02544,0.17446,-0.46849,0.02288,3108.0,8.80,414.0,0.016,-6.68,NaN,32.01,-0.044775,0.0,0,0
3,420.68,4.06,0.69735,0.02335,0.16063,-0.44086,0.02317,1070.0,10.93,411.0,0.023,-9.68,NaN,32.01,-0.044775,0.0,0,0
4,423.38,4.04,0.63464,0.02711,0.17433,-0.43781,0.02282,7502.0,8.77,417.0,0.015,-6.38,NaN,30.86,-0.044775,0.0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2895,433.31,2.21,0.84058,0.02339,0.08571,-0.34777,0.01188,80.0,14.39,419.0,0.033,-14.31,2.0,27.58,0.089130,0.0,0,0
2896,433.31,2.21,0.83081,0.02509,0.08944,-0.35051,0.01248,413.0,13.48,420.0,0.031,-13.31,3.0,27.58,0.089130,0.0,0,0
2897,433.31,2.21,0.69645,0.03227,0.12836,-0.52916,0.01394,1706.0,8.71,426.0,0.017,-7.31,3.0,27.58,0.089130,0.0,0,0
2898,433.31,2.21,0.78689,0.02699,0.10522,-0.43651,0.01315,174.0,11.82,422.0,0.026,-11.31,3.0,27.58,0.089130,0.0,0,0


In [20]:
df.to_csv("../Resources/test_ready.csv", index=True)

# __*STOP HERE AND CON'T TO "forests_no_random_splits.ipynb"*__

# SECTION 3: Evaluate Algorithms and Models

## 3.1.) Train Test Split and Evaluation Metrics

In [21]:
#Split into training and testing groups

#Split out validation dataset for the end
y = df["y"]

X = df.drop(columns='y')


In [22]:
y.value_counts()

1.0    587
5.0    528
2.0    351
3.0    334
4.0    181
0.0    105
Name: y, dtype: int64

In [23]:

validation_size = 0.2

seed = 7

X_train, X_validation, y_train, y_validation = train_test_split(X, y, test_size=validation_size, stratify=y, random_state=seed)

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [ ]:
X_validation

In [ ]:
# Create the StandardScaler instance
scaler = StandardScaler()

# Fit the scaler to the features training dataset
X_scaler = scaler.fit(X_train)

# Scale both the training and testing data from the features dataset
X_train_scaled = X_scaler.transform(X_train)
X_validation_scaled = X_scaler.transform(X_validation)

## 3.2.) Quick Check of Models and Algorithms

In [ ]:
#Spot check Classification algorithms

models = []

#Boosting methods

models.append(('XGB', XGBClassifier()))
models.append(('AB', AdaBoostClassifier()))
models.append(('GBM', GradientBoostingClassifier()))

#Bagging methods

models.append(('RF', RandomForestClassifier()))
models.append(('ET', ExtraTreesClassifier()))

#SVC 
models.append(('SVC', SVC()))

In [ ]:
#Test options for classification

num_folds = 10
seed = 7

In [ ]:
results = []

names = []

scoring = 'accuracy'

for name, model in models:
    kfold = StratifiedKFold(n_splits=num_folds, random_state=seed, shuffle=True)
    cv_results = cross_val_score(model, X_train_scaled, y_train, cv=kfold, scoring=scoring)
    results.append(cv_results)
    names.append(name)
    findings_summary = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
    print(findings_summary)

In [ ]:
#Compare the model outcomes

fig = pyplot.figure()
fig.suptitle('Model Comparison')
ax = fig.add_subplot(111)
pyplot.boxplot(results)
ax.set_xticklabels(names)
fig.set_size_inches(8,4)
pyplot.show()

## 3.3.) Selecting Random Forest (or whichver tree method best), Calculating its Baseline 

In [ ]:
#Estimate accuracy on training set

rf_model = RandomForestClassifier()

cv = StratifiedKFold(n_splits=num_folds, random_state=seed, shuffle=True)
scores = cross_val_score(rf_model, X_train_scaled, y_train, scoring='accuracy', cv=cv)
avg_score = np.mean(scores)
avg_score

In [ ]:
#Estimate accuracy on validation set

rf_model = RandomForestClassifier()

rf_model.fit(X_train_scaled, y_train)

predictions = rf_model.predict(X_validation_scaled)

print(accuracy_score(y_validation, predictions))

print('Weighted Precision: {:.2f}'.format(precision_score(y_validation, predictions, average='weighted')))
print('Weighted Recall: {:.2f}'.format(recall_score(y_validation, predictions, average='weighted')))
print('Weighted F1-score: {:.2f}'.format(f1_score(y_validation, predictions, average='weighted')))

print(classification_report(y_validation, predictions, target_names=["Strong Sell", "Sell", "Pass", "Buy", "Strong Buy", "Very High Return"]))

print(confusion_matrix(y_validation, predictions))

In [ ]:
#Visualization of the Confusion Matrix

df_matrix = pd.DataFrame(confusion_matrix(y_validation, predictions), columns=np.unique(y_validation), index = np.unique(y_validation))
df_matrix.index.name = 'Actual'
df_matrix.columns.name = 'Predicted'
sns.heatmap(df_matrix, cmap="Blues", annot=True, annot_kws={"size": 16}) 

# SECTION 4: Model Tuning 

## 4.1.) Hyperparameter Tuning for Random Forest

### APPROACH (A) 
## __*(CAUTION TAKES A LONG TIME!)*__

In [ ]:


# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]

# Number of features to consider at every split
max_features = ['auto', 'sqrt']

# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)

# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]

# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]

# Method of selecting samples for training each tree
bootstrap = [True, False]

# Create the random grid
random_grid = {}

paramgrid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,}
              # 'min_samples_split': min_samples_split,
               #'min_samples_leaf': min_samples_leaf,
              # 'bootstrap': bootstrap}

random.seed(1)

cv = EvolutionaryAlgorithmSearchCV(estimator=RandomForestClassifier(),
                                   params=paramgrid,
                                   scoring="accuracy",
                                   cv=StratifiedKFold(n_splits=4),
                                   verbose=1,
                                   population_size=50,
                                   gene_mutation_prob=0.10,
                                   gene_crossover_prob=0.5,
                                   tournament_size=3,
                                   generations_number=5,
                                   n_jobs=4)
cv.fit(X_train_scaled, y_train)

### APPROACH (B)
## __*(CAUTION TAKES A LONG TIME!)*__

In [ ]:
# Grid Search: (select model) Tuning

n_estimators = [20, 100, 180, 1000]
max_features = [1, 2, 3, 4]

param_grid = dict(n_estimators=n_estimators, max_features=max_features)

rf_model = RandomForestClassifier()

kfold = StratifiedKFold(n_splits=num_folds, random_state=seed, shuffle=True)

grid = GridSearchCV(estimator=rf_model, param_grid=param_grid, scoring=scoring, cv=kfold)
grid_result = grid.fit(X_train_scaled, y_train)

#Print Results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
ranks = grid_result.cv_results_['rank_test_score']
for mean, stdev, param, rank in zip(means, stds, params, ranks):
    print("#%d %f (%f) with: %r" % (rank, mean, stdev, param))

### PROCEEDING WITH BEST APPROACH

In [ ]:
#Prepare the model with parameters decided in previous cell

rf_model = RandomForestClassifier() #(n_estimators=200, max_features="sqrt", max_depth=100, min_samples_split=5, min_samples_leaf=1, bootstrap=False)
rf_model.fit(X_train_scaled, y_train)

In [ ]:
#Score predictions of training set

training_predictions = rf_model.predict(X_train_scaled)

print(accuracy_score(y_train, training_predictions))

print('Micro Precision: {:.2f}'.format(precision_score(y_train, training_predictions, average='micro')))
print('Micro Recall: {:.2f}'.format(recall_score(y_train, training_predictions, average='micro')))
print('Micro F1-score: {:.2f}\n'.format(f1_score(y_train, training_predictions, average='micro')))

print('Macro Precision: {:.2f}'.format(precision_score(y_train, training_predictions, average='macro')))
print('Macro Recall: {:.2f}'.format(recall_score(y_train, training_predictions, average='macro')))
print('Macro F1-score: {:.2f}\n'.format(f1_score(y_train, training_predictions, average='macro')))

print('Weighted Precision: {:.2f}'.format(precision_score(y_train, training_predictions, average='weighted')))
print('Weighted Recall: {:.2f}'.format(recall_score(y_train, training_predictions, average='weighted')))
print('Weighted F1-score: {:.2f}'.format(f1_score(y_train, training_predictions, average='weighted')))


print(classification_report(y_train, training_predictions, target_names=["Strong Sell", "Sell", "Pass", "Buy", "Strong Buy", "Very High Return"]))

In [ ]:
#Score predictions of validation set

predictions = rf_model.predict(X_validation_scaled)

print(accuracy_score(y_validation, predictions))

print('Micro Precision: {:.2f}'.format(precision_score(y_validation, predictions, average='micro')))
print('Micro Recall: {:.2f}'.format(recall_score(y_validation, predictions, average='micro')))
print('Micro F1-score: {:.2f}\n'.format(f1_score(y_validation, predictions, average='micro')))

print('Macro Precision: {:.2f}'.format(precision_score(y_validation, predictions, average='macro')))
print('Macro Recall: {:.2f}'.format(recall_score(y_validation, predictions, average='macro')))
print('Macro F1-score: {:.2f}\n'.format(f1_score(y_validation, predictions, average='macro')))

print('Weighted Precision: {:.2f}'.format(precision_score(y_validation, predictions, average='weighted')))
print('Weighted Recall: {:.2f}'.format(recall_score(y_validation, predictions, average='weighted')))
print('Weighted F1-score: {:.2f}'.format(f1_score(y_validation, predictions, average='weighted')))


print(classification_report(y_validation, predictions, target_names=["Strong Sell", "Sell", "Pass", "Buy", "Strong Buy", "Very High Return"]))

print(confusion_matrix(y_validation, predictions))

In [ ]:
#Visualization of the Confusion Matrix

df_matrix = pd.DataFrame(confusion_matrix(y_validation, predictions), columns=np.unique(y_validation), index = np.unique(y_validation))
df_matrix.index.name = 'Actual'
df_matrix.columns.name = 'Predicted'
sns.heatmap(df_matrix, cmap="Blues", annot=True, annot_kws={"size": 16}) 

## 4.2.) Determining Feature Importance

In [ ]:
#Feature importances

importances = rf_model.feature_importances_

In [ ]:
important_features = zip(X.columns, rf_model.feature_importances_)

In [ ]:
importances_df = pd.DataFrame(important_features)

In [ ]:
importances_df = importances_df.rename(columns={0: 'Feature', 1: 'Importance'})

In [ ]:
importances_df = importances_df.set_index('Feature')

In [ ]:
importances_df = importances_df.sort_values(by='Importance', ascending=False)

In [ ]:
importances_df

In [ ]:
#Plot the top 10 most important features

importances_df[0:10].plot(
    kind='barh',
    color='green',
    title='Feature Importance',
    legend=True)

## 4.2. Bayesian Optimization with HYPEROPT for XGB

## __*(CAUTION TAKES A COUPLE OF MIN!)*__

In [ ]:
#Initialize domain space for range of values 
 
space={'max_depth': hp.quniform("max_depth", 3, 18, 1),
        'gamma': hp.uniform ('gamma', 1, 9),
        'reg_alpha' : hp.quniform('reg_alpha', 40, 180, 1),
        'reg_lambda' : hp.uniform('reg_lambda', 0, 1),
        'colsample_bytree' : hp.uniform('colsample_bytree', 0.5, 1),
        'min_child_weight' : hp.quniform('min_child_weight', 0, 10, 1),
        'n_estimators': 180,
        'seed': 0
    }


The available hyperopt optimization algorithms are -

hp.choice(label, options) — Returns one of the options, which should be a list or tuple.

hp.randint(label, upper) — Returns a random integer between the range [0, upper).

hp.uniform(label, low, high) — Returns a value uniformly between low and high.

hp.quniform(label, low, high, q) — Returns a value round(uniform(low, high) / q) * q, i.e it rounds the decimal values and returns an integer.

hp.normal(label, mean, std) — Returns a real value that’s normally-distributed with mean and standard deviation sigma.

In [ ]:
def objective(space):
    xgb_model = xgb.XGBClassifier(
                    n_estimators =space['n_estimators'], max_depth = int(space['max_depth']), gamma = space['gamma'],
                    reg_alpha = int(space['reg_alpha']),min_child_weight=int(space['min_child_weight']),
                    colsample_bytree=int(space['colsample_bytree']))
    
    evaluation = [(X_train_scaled, y_train), (X_validation_scaled, y_validation)]
    
    xgb_model.fit(X_train_scaled, y_train,
            eval_set=evaluation, eval_metric="auc",
            early_stopping_rounds=10,verbose=False)
    

    pred = xgb_model.predict(X_validation_scaled)
    accuracy = accuracy_score(y_validation, pred)
    print ("SCORE:", accuracy)
    return {'loss': -accuracy, 'status': STATUS_OK }

In [ ]:
trials = Trials()

best_hyperparams = fmin(fn = objective,
                        space = space,
                        algo = tpe.suggest,
                        max_evals = 100,
                        trials = trials)

In [ ]:
print("The best hyperparameters are : ","\n")
print(best_hyperparams)

In [ ]:
#Establish XGB model instance
#Fitting training dataset to model

xgb_model = xgb.XGBClassifier(colsample_bytree=0.7417632345631163, max_depth=8, gamma=1.5654068039190379, min_child_weight=3.0, reg_lambda=0.8585451920481999, reg_alpha=0)
xgb_model.fit(X_train_scaled, y_train)

In [ ]:
#Predicting on training set

training_predictions = xgb_model.predict(X_train_scaled)

print(accuracy_score(y_train, training_predictions))

print('Weighted Precision: {:.2f}'.format(precision_score(y_train, training_predictions, average='weighted')))
print('Weighted Recall: {:.2f}'.format(recall_score(y_train, training_predictions, average='weighted')))
print('Weighted F1-score: {:.2f}'.format(f1_score(y_train, training_predictions, average='weighted')))

print(classification_report(y_train, training_predictions, target_names=["Strong Sell", "Sell", "Pass", "Buy", "Strong Buy", "Very High Return"]))

In [ ]:
#Predicting on validation set

predictions = xgb_model.predict(X_validation_scaled)

print(accuracy_score(y_validation, predictions))

print('Weighted Precision: {:.2f}'.format(precision_score(y_validation, predictions, average='weighted')))
print('Weighted Recall: {:.2f}'.format(recall_score(y_validation, predictions, average='weighted')))
print('Weighted F1-score: {:.2f}'.format(f1_score(y_validation, predictions, average='weighted')))

print(classification_report(y_validation, predictions, target_names=["Strong Sell", "Sell", "Pass", "Buy", "Strong Buy", "Very High Return"]))

print(confusion_matrix(y_validation, predictions))

In [ ]:
#Get probabilities of predictions

pred_proba = xgb_model.predict_proba(X_validation_scaled)
pred_proba

In [ ]:
proba_df = pd.DataFrame(pred_proba.round(2))

In [ ]:
#Formatting probabilities DataFrame

proba_df["Actual"] = y_validation.reset_index(drop=True)
proba_df.columns = ["'Strong Sell' Probability", "'Sell' Probability", "'Pass' Probability", "'Buy' Probability", "'Strong Buy' Probability", "'Very High Return' Prob.", "Actual"]
proba_df

In [ ]:
#Labeling caterogires

proba_df.loc[proba_df["Actual"] == 0, "Actual"] = "Strong Sell"
proba_df.loc[proba_df["Actual"] == 1, "Actual"] = "Sell"
proba_df.loc[proba_df["Actual"] == 2, "Actual"] = "Pass"
proba_df.loc[proba_df["Actual"] == 3, "Actual"] = "Buy"
proba_df.loc[proba_df["Actual"] == 4, "Actual"] = "Strong Buy"
proba_df.loc[proba_df["Actual"] == 5, "Actual"] = "Very High Return"

proba_df.head(50)

In [ ]:
#Get ROC AUC score

roc_auc_score(y_validation, pred_proba, multi_class="ovr")

## 4.2.) Determining Feature Importance for XGBoost Model

In [ ]:
#Feature importances

importances = xgb_model.feature_importances_

In [ ]:
important_features = zip(X.columns, xgb_model.feature_importances_)

In [ ]:
importances_df = pd.DataFrame(important_features)

In [ ]:
importances_df = importances_df.rename(columns={0: 'Feature', 1: 'Importance'})

In [ ]:
importances_df = importances_df.set_index('Feature')

In [ ]:
importances_df = importances_df.sort_values(by='Importance', ascending=False)

In [ ]:
importances_df

In [ ]:
#Plot the top 10 most important features

importances_df[0:10].plot(
    kind='barh',
    color='green',
    title='Feature Importance',
    legend=True)

In [ ]:
#et_model = ExtraTreesClassifier()
#et_model.fit(X_train_scaled, y_train)

In [ ]:
#ab_model = AdaBoostClassifier()
#ab_model.fit(X_train_scaled, y_train)

In [ ]:
#gb_model = GradientBoostingClassifier()
#ab_model.fit(X_train_scaled, y_train)